In [1]:
import os
os.environ['PYTHONHASHSEED'] = '0'
import random as rn
rn.seed(12345)
from numpy.random import seed
seed(42)
from tensorflow.compat.v1 import set_random_seed
set_random_seed(42)

import yaml
import datetime
import gc

import nipy as ni
import nibabel as nib
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import load_model
from GPUtil import getFirstAvailable
from tensorflow.keras import backend as K
from nibabel.streamlines.trk import TrkFile
from nibabel.streamlines.array_sequence import ArraySequence, concatenate
from nibabel.streamlines.tractogram import Tractogram
from hashlib import md5
from sklearn.preprocessing import normalize
from pathos.multiprocessing import Pool
from scipy.interpolate import RegularGridInterpolator
from time import time

import tensorflow_probability as tfp

import warnings
warnings.filterwarnings('ignore')

In [2]:
def predict(config):
    """
    seeds: [[x1,y1,z1],...,[xn,yn,zn]]
    """
    # Load Data
    
    subject_dir = os.path.join("..", "subjects", config["subject"])
    
    hasher = md5()
    for v in config.values():
        hasher.update(str(v).encode())
    
    save_dir = os.path.join(subject_dir, "predicted_fibers", hasher.hexdigest())
    if os.path.exists(save_dir):
        print("Predictions with this config have been created already:\n{}".format(save_dir))
        return
    
    print("Loading DWI...")
    
    dwi_path = os.path.join(subject_dir, config["dwi_path"])
    dwi_img = nib.load(dwi_path)
    dwi_aff1 = dwi_img.affine

    dwi_img = nib.funcs.as_closest_canonical(dwi_img)
    dwi_aff2 = dwi_img.affine
    dwi_affi2 = np.linalg.inv(dwi_aff2)
    dwi = dwi_img.get_data()
    
    print("Loading Model...")
    
    model_path = os.path.join(config["model_dir"], "model.h5")
    
    def negative_log_likelihood(observed_y, predicted_distribution):
        return -K.mean(predicted_distribution.log_prob(observed_y))
    
    model = load_model(model_path,
                       custom_objects={"negative_log_likelihood": negative_log_likelihood,
                                       "DistributionLambda": tfp.layers.DistributionLambda})
    
    # Define coordinate transforms
    
    input_shape = model.layers[0].get_output_at(0).get_shape().as_list()[-1]
    block_size = int(np.cbrt(input_shape / dwi.shape[-1]))
    
    def xyz2ijk(coords, snap=False, shift=False):
        ijk = dwi_affi2.dot(coords.T).T
        if snap:
            ijk = np.round(ijk).astype(int)
        return ijk
    
    # Define Fiber Termination
    
    class Terminator(object):
        def __init__(self):
            if ".nii" in config["terminator"][0]:
                scalar_img = nib.load(os.path.join(subject_dir, config["terminator"][0]))
                self.scalar = scalar_img.get_data()
                affi = np.linalg.inv(scalar_img.affine)
                self.xyz2ijk = lambda xyz: np.round(affi.dot(xyz.T).T).astype(int)
            else:
                raise NotImplementedError # TODO: Implement termination model
            self.threshold = config["terminator"][1]
        def __call__(self, xyz):
            if hasattr(self, "scalar"):
                ijk = self.xyz2ijk(xyz)
                return np.where(self.scalar[ijk[:,0], ijk[:,1], ijk[:,2]] < self.threshold)[0]
            else:
                raise NotImplementedError

    terminator = Terminator()
    
    print("Loading Seeds...")
    
    seed_file = nib.streamlines.load(config["seed_path"])
    seeds = seed_file.tractogram.streamlines.data
    seeds = np.vstack([seeds, seeds])  # Duplicate seeds for positive and negative starting direction
    seeds = np.hstack([seeds, np.ones([len(seeds), 1])]) # add affine dimension
    assert seeds.shape[-1] == 4   # (x, y, z, 1)
    
    # Define Prior for First Fiber Direction
    
    class Prior(object):
        def __init__(self):
            if ".nii" in config["prior"]:
                peak_img = nib.load(os.path.join(subject_dir, config["prior"]))
                self.peak = peak_img.get_data()
                affi = np.linalg.inv(peak_img.affine)
                self.xyz2ijk = lambda xyz: np.round(affi.dot(xyz.T).T).astype(int)
            elif ".h5" in config["prior"]:
                raise NotImplementedError # TODO: Implement prior model
                
        def __call__(self, xyz):
            if hasattr(self, "peak"):
                ijk = self.xyz2ijk(xyz)
                # Assuming that seeds have been duplicated
                peaks = self.peak[ijk[:,0], ijk[:,1], ijk[:,2]]
                peaks[len(ijk)//2:, :] *= -1
                return normalize(peaks)
            elif hasattr(self, "model"):
                raise NotImplementedError # TODO: Implement prior model
        
    prior = Prior()
    
    print("Initialize Fibers...")
    
    xyz = seeds.reshape(-1, 1, 4) # (fiber, segment, coord) we assume seeds to be in rasmm!
    
    fiber_idx = np.hstack([np.arange(len(seeds)//2), np.arange(len(seeds)//2)])
    fibers = [[] for _ in range(len(seeds)//2)]
    
    print("Start Iteration...")
    
    for i in range(config["max_steps"]):
        t0 = time()
        
        ijk = xyz2ijk(xyz[:,-1,:], snap=True, shift=True) # Get coords of latest segement for each fiber 

        d = np.zeros([len(ijk), block_size, block_size, block_size, dwi.shape[-1]])

        for ii, idx in enumerate(ijk):
            d[ii] = dwi[idx[0] - (block_size // 2) : idx[0] + (block_size // 2) + 1,
                        idx[1] - (block_size // 2) : idx[1] + (block_size // 2) + 1,
                        idx[2] - (block_size // 2) : idx[2] + (block_size // 2) + 1,
                    :]
        d = d.reshape(-1, dwi.shape[-1] * block_size**3)
        
        if i == 0:
            vin = prior(xyz[:,0,:])
        else:
            vin = vout.copy()
        
        chunk_size = 2**15 # 32768
        n_chunks = np.ceil(len(vin) / chunk_size).astype(int)
        
        inputs = np.hstack([vin,d])
        vout = np.zeros([len(vin), 3])
        for chunk in range(n_chunks):
            input_chunk = inputs[chunk * chunk_size : (chunk + 1) * chunk_size]
            if config["predict_fn"] == "mean":
                v = model(input_chunk).mean().numpy()
                v = normalize(v) # Careful, the FvM mean is not a unit vector!
            else:
                v = model(input_chunk).sample().numpy() # Samples are unit length, though!
            vout[chunk * chunk_size : (chunk + 1) * chunk_size] = v
           
        rout = (xyz[:, -1, :3] + config["step_size"] * vout)
        rout = np.hstack([rout, np.ones((len(rout), 1))]).reshape(-1, 1, 4)
        
        xyz = np.concatenate([xyz, rout], axis=1)
        
        terminal_indices = terminator(xyz[:, -1, :]) # Check latest points for termination

        for idx in terminal_indices:
            gidx = fiber_idx[idx]
            # Other end not yet added
            if not fibers[gidx]:
                fibers[gidx].append(xyz[idx, :, :3])
            # Other end already added
            else:
                this_end = xyz[idx, :, :3]
                other_end = fibers[gidx][0]
                merged_fiber = np.vstack([np.flip(this_end[1:], axis=0), other_end]) # stitch ends together
                fibers[gidx] = [merged_fiber]
                
        xyz = np.delete(xyz, terminal_indices, axis=0)
        vout = np.delete(vout, terminal_indices, axis=0)
        fiber_idx = np.delete(fiber_idx, terminal_indices)
        
        print("Iter {:4d}/{}, finished {:5d}/{:5d} ({:3.0f}%) of all seeds with {:6.0f} steps/sec".format(
            (i+1), config["max_steps"], len(seeds)-len(fiber_idx), len(seeds),
            100*(1-len(fiber_idx)/len(seeds)), len(vin) / (time() - t0)),
            end="\r"
        )
        
        if len(fiber_idx) == 0:
            break
            
        gc.collect()
    
    # Include unfinished fibers:
    
    for idx, gidx in enumerate(fiber_idx):
        if not fibers[gidx]:
            fibers[gidx].append(xyz[idx, :, :3])
        else:
            this_end = xyz[idx, :, :3]
            other_end = fibers[gidx][0]
            merged_fiber = np.vstack([np.flip(this_end[1:], axis=0), other_end])
            fibers[gidx] = [merged_fiber]
    
    # Save Result
    
    fibers = [f[0] for f in fibers]
    
    tractogram = Tractogram(
        streamlines=ArraySequence(fibers),
        affine_to_rasmm=np.eye(4)
    )
    
    # tractogram.apply_affine(affine_original.dot(affine_canonical_inv))
    
    os.makedirs(save_dir, exist_ok=True)

    fiber_path = os.path.join(save_dir, "fibers.trk")
    print("\nSaving {}".format(fiber_path))
    TrkFile(tractogram, seed_file.header).save(fiber_path)
    #nib.streamlines.save(tractogram, fiber_path)

    config_path = os.path.join(save_dir, "config.yml")
    print("Saving {}".format(config_path))
    with open(config_path, "w") as file:
        yaml.dump(config, file, default_flow_style=False) 
    
    return tractogram

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(getFirstAvailable(order="load", maxLoad=10**-6, maxMemory=10**-1)[0])

In [4]:
config = dict(
    subject="ismrm_gt",
    model_dir="../models/entrack_conditional/f57290355caf38c41d6aeca1c7dc2091",
    dwi_path="fod_norm_125.nii.gz",
    prior="tensor_V1_125.nii.gz",
    seed_path="../scoring/scoring_data/seeds/seeds_from_wm_100.trk",
    terminator=["wm_mask_125.nii.gz", 0.1],
    predict_fn="mean",
    step_size=0.25,
    max_steps=400
)

In [5]:
tractogram = predict(config)

Loading DWI...
Loading Model...


W1024 08:29:26.110945 140356650682112 deprecation.py:323] From /local/home/vwegmayr/miniconda2/envs/thesis/lib/python3.6/site-packages/tensorflow_probability/python/distributions/von_mises_fisher.py:312: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Loading Seeds...
Initialize Fibers...
Start Iteration...


KeyboardInterrupt: 